In [ ]:
% matplotlib inline

import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import gsw as sw
from scipy.spatial.distance import pdist, squareform
from statsmodels.tsa.tsatools import detrend


from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset as NetCDFFile

In [ ]:
## this will all be saved into external python files once I have it working
def SVh( P, h, bw ):
    '''
    Experimental semivariogram for a single lag
    '''
    pd = squareform( pdist( P[:,:2] ) )
    N = pd.shape[0]
    Z = list()
    for i in range(N):
        for j in range(i+1,N):
            if( pd[i,j] >= h-bw )and( pd[i,j] <= h+bw ):
                Z.append( ( P[i,2] - P[j,2] )**2.0 )
    return np.sum( Z ) / ( 2.0 * len( Z ) )
 
def SV( P, hs, bw ):
    '''
    Experimental variogram for a collection of lags
    '''
    sv = list()
    for h in hs:
        sv.append( SVh( P, h, bw ) )
    sv = [ [ hs[i], sv[i] ] for i in range( len( hs ) ) if sv[i] > 0 ]
    return np.array( sv ).T
 
def C( P, h, bw ):
    '''
    Calculate the sill
    '''
    c0 = np.var( P[:,2] )
    if h == 0:
        return c0
    return c0 - SVh( P, h, bw )

def opt( fct, x, y, C0, parameterRange=None, meshSize=1000 ):
    if parameterRange == None:
        parameterRange = [ x[1], x[-1] ]
    mse = np.zeros( meshSize )
    a = np.linspace( parameterRange[0], parameterRange[1], meshSize )
    for i in range( meshSize ):
        mse[i] = np.mean( ( y - fct( x, a[i], C0 ) )**2.0 )
    return a[ mse.argmin() ]

def spherical(h, a, C0):
    '''
    Spherical model of the semivariogram
    '''
    # if h is a single digit
    if type(h) == np.float64:
        # calculate the spherical function
        if h <= a:
            return C0*(1.5*(h/a) - (1/3)*((h**3.0)/(a**3.0)))
        else:
            return C0
    # if h is an iterable
    else:
        # calculate the spherical function for all elements
        a = np.ones( h.size ) * a
        C0 = np.ones( h.size ) * C0
        return map(spherical, h, a, C0)
    
def cvmodel(P, model, hs, bw):
    '''
    Input:  (P)      ndarray, data
            (model)  modeling function
                      - spherical
                      - exponential
                      - gaussian
            (hs)     distances
            (bw)     bandwidth
    Output: (covfct) function modeling the covariance
    '''
    # calculate the semivariogram
    sv = SV( P, hs, bw )
    # calculate the sill
    C0 = C(P, hs[0], bw)
    # calculate the optimal parameters
    param = opt( model, sv[0], sv[1], C0 )
    # return a covariance function
    covfct = lambda h, a=param: C0 - model( h, a, C0 )
    return covfct

In [ ]:
# import data 

filePath = r'/Users/sclayton/Google Drive/seaflow_data/armbrustlab-seaflow-tot_chl_byfile.csv'
result = pd.read_csv(filePath)

# calculate distance between points

In [ ]:
# bin the data into 2 degree bins, and calculate average in each bin (do for each cruise separately)
# remove the mean from the data points. 

# bin all data into 2x2 degree bins and get mean and std

binsize = 2
xbin = np.arange(120, 340, binsize)
ybin = np.arange(10, 50, binsize)

xgrid, ygrid = np.meshgrid(xbin, ybin, sparse=False, indexing='ij')

var = []
binned = np.empty([len(xbin), len(ybin), len(var)])
binnedstd = np.empty([len(xbin), len(ybin), len(var)])
numobs = np.empty([len(xbin), len(ybin)])
nummonth = np.empty([len(xbin), len(ybin)])

for i in range(len(xbin)):
    lon = xbin[i]
    for j in range(len(ybin)):
        lat = ybin[j]
        
        for k in range(len(var)):
            fvar = var[k]
            nn = len(data.loc[((data['lon_e']>lon-binsize/2) & (data['lon_e']<lon +binsize/2) & (data['lat']>lat -binsize/2)
                                              & (data['lat']<lat +binsize/2)), fvar].values)
            numobs[i,j] = nn
            nummonth[i,j] = len(pd.unique(data.loc[((data['lon_e']>lon-binsize/2) & (data['lon_e']<lon +binsize/2) & (data['lat']>lat -binsize/2)
                                              & (data['lat']<lat +binsize/2)), 'month'].values))
            if nn > 0:
                binned[i,j,k] = np.nanmean(data.loc[((data['lon_e']>lon-binsize/2) & (data['lon_e']<lon +binsize/2) & (data['lat']>lat - binsize/2)
                                              & (data['lat']<lat + binsize/2)), fvar].values)
                binnedstd[i,j,k] = np.nanstd(data.loc[((data['lon_e']>lon-binsize/2) & (data['lon_e']<lon +binsize/2) & (data['lat']>lat - binsize/2)
                                              & (data['lat']<lat + binsize/2)), fvar].values)

            else:
                binned[i,j,k] = np.nan
                binnedstd[i,j,k] = np.nan
                numobs[i,j] = np.nan
                nummonth[i,j] = np.nan


# calculate semivariogram for each bin
# bandwidth, plus or minus 0.5 km
bw = 2
# lags in 500 meter increments from zero to 200km
hs = np.arange(0,101,bw)
sv = SV( P, hs, bw )

# fit the model to the semivariogram
sp = cvmodel( P, model=spherical, hs=hs, bw=bw )
plt.plot( sv[0], sv[1], '.' )
plt.plot( sv[0], sp( sv[0] ) ) ;
plt.title('Spherical Model')
plt.ylabel('Semivariance')
plt.xlabel('Lag [m]')
plt.show()